# Strawberry

## Libraries dependency and imports

In [6]:
!pip install strawberry-graphql
!pip install uvicorn[standard]
!pip install fastapi
!pip install psycopg2-binary

## Helper Func for App in Notebook

V ukázkách dále bude použit kód, který je specifický pro prostředí jupyter a který tak umožňuje spouštět ukázky přímo v notebooku. Fakticky je kódem vytvořen subproces, který zabezpečuje běh serveru. Identifikace subprocesu je uložena v datové struktuře `servers`. Díky tomu lze identifikovat, zda na požadovaném portu již nějaký server běží a v případě potřeby jej zastavit a spustit nový server.

Po ukončení experimentů se serverem (kódem) je nutné tento server zastavit, aby došlo k uvolnění portu. V případe problémů je možné, že bude nezbytné restartovat jupyter, aby byly porty uvolněny. Je-li spuštěn nový server, aniž by běžící na stejném portu byl ukončen, dojde k chybovému stavu.

```python
assert port in [9991, 9992, 9993, 9994]
```
Slouží k ověření, že požadovaný port je dostupný i z prostředí mimo jupyter. Vzpomeňte si na konfiguraci docker stacku a mapování portů mimo jupyter kontejner.

In [1]:
# Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
# Just ignore it!
import uvicorn
from multiprocessing import Process

servers = {}

def start_api(app=None, port=9992, runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    assert port in [9991, 9992, 9993, 9994], f'port has unexpected value {port}'
    def run():
        uvicorn.run(app, port=port, host='0.0.0.0', root_path='')    
        
    _api_process = servers.get(port, None)
    if _api_process:
        _api_process.terminate()
        _api_process.join()
        del servers[port]
    
    if runNew:
        assert (not app is None), 'app is None'
        _api_process = Process(target=run, daemon=True)
        _api_process.start()
        servers[port] = _api_process

In [3]:
import os
import asyncio
import multiprocessing
servers = {}

def _start_api(app=None, port=9992, runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    assert port in [9991, 9992, 9993, 9994], f'port has unexpected value {port}'
    
    async def runAsync():
        config = uvicorn.Config(app, port=port, host='0.0.0.0', log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
        #uvicorn.run(app, port=port, host='0.0.0.0', root_path='')    
        
    def withLoop():
        process_name = "[Process %s]" % (os.getpid())
        print("%s Started " % process_name)

        #loop = asyncio.get_event_loop()
        loop = asyncio.new_event_loop()
        try:
            loop.run_until_complete(runAsync())
        except KeyboardInterrupt:
            print("%s Loop interrupted" % process_name)
            loop.stop()

        print("%s terminating" % process_name)
        pass
    
    _api_process = servers.get(port, None)
    if _api_process:
        _api_process.terminate()
        _api_process.join()
        del servers[port]
    
    if runNew:
        assert (not app is None), 'app is None'
        _api_process = multiprocessing.Process(target=withLoop)
        #_api_process = Process(target=run, daemon=True)
        _api_process.start()
        servers[port] = _api_process

In [8]:
!pip install gunicorn

? # Code in this cell is just for (re)starting the API on a Process, and other compatibility stuff with Jupyter cells.
? # Just ignore it!
import uvicorn
from multiprocessing import Process

async def start_api(app=None, port=9992, runNew=True):
    """Stop the API if running; Start the API; Wait until API (port) is available (reachable)"""
    assert port in [9991, 9992, 9993, 9994], f'port has unexpected value {port}'

    config = uvicorn.Config(app, port=port, host='0.0.0.0', log_level="info")
    server = uvicorn.Server(config)
    await server.serve()

await start_api(app, port=9992, runNew=True)

In [31]:
#start_api(app, port=9992, runNew=False)

## Hello World in Strawberry

Následující kód je "klasický" hello world, po spuštění otevřete 

http://localhost:31102/gql

Přestože je výše v kódu definovaný port 9992, je nutné si uvědomit, že ve stacku learning je tento port mapován na 31102.
Pokud máte jiný stack, či jiné prostředí, přizpůsobte si url adresu.

Neopomeňte server ukončit `start_api(runNew=False)`

In [9]:
import strawberry
import uuid

@strawberry.type(description="""Type for query root""")
class Query:

    @strawberry.field(description="""Returns a hello""")
    async def say_hello(self, info: strawberry.types.Info, id: strawberry.ID) -> str:
        result = f'Hello {id}'
        return result
    
from strawberry.asgi import GraphQL

graphql_app = GraphQL(
    strawberry.federation.Schema(Query), 
    graphiql = True,
    allow_queries_via_get = True
)

from fastapi import FastAPI
app = FastAPI()
app.mount("/gql", graphql_app)

start_api(app, port=9992, runNew=True)

INFO:     Started server process [9173]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)


INFO:     172.18.0.1:54098 - "GET /gql HTTP/1.1" 307 Temporary Redirect
INFO:     172.18.0.1:54100 - "GET / HTTP/1.1" 200 OK
INFO:     172.18.0.1:54100 - "POST / HTTP/1.1" 200 OK
INFO:     172.18.0.1:54128 - "POST / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9173]


http://localhost:31102/gql

In [10]:
start_api(app, port=9992, runNew=False)

## Strawberry with objects

In [11]:
import strawberry
import uuid

@strawberry.federation.type(keys=["id"], description="""Entity representing an user""")
class UserGQLModel:
    @classmethod
    async def resolve_reference(cls, info: strawberry.types.Info, id: strawberry.ID):
        result = {'name': f'John({id})', 'id': id}
        return result
    
    @strawberry.field(description="""user name""")
    async def name(self, info: strawberry.types.Info) -> str:
        result = self['name']
        return result
    
    @strawberry.field(description="""user id""")
    async def id(self, info: strawberry.types.Info) -> str:
        result = self['id']
        return result
    
@strawberry.type(description="""Type for query root""")
class Query:

    @strawberry.field(description="""Returns a hello""")
    async def user_by_id(self, info: strawberry.types.Info, id: strawberry.ID) -> UserGQLModel:
        result = await UserGQLModel.resolve_reference(info, id)
        return result
    
from strawberry.asgi import GraphQL

graphql_app = GraphQL(
    strawberry.federation.Schema(Query), 
    graphiql = True,
    allow_queries_via_get = True
)

from fastapi import FastAPI
app = FastAPI()
app.mount("/gql", graphql_app)

start_api(app, port=9992, runNew=True)

INFO:     Started server process [10967]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)


INFO:     172.18.0.1:54678 - "POST / HTTP/1.1" 200 OK
INFO:     172.18.0.1:54732 - "POST / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [10967]


In [17]:
start_api(app, port=9992, runNew=False)

In [2]:
import strawberry
import uuid

@strawberry.federation.type(keys=["id"], description="""Entity representing an user""")
class UserGQLModel:
    @classmethod
    async def resolve_reference(cls, info: strawberry.types.Info, id: strawberry.ID):
        result = {'name': f'John({id})', 'id': id, 'surname': 'Newbie'}
        return result
    
    @strawberry.field(description="""user name""")
    async def name(self, info: strawberry.types.Info) -> str:
        result = self['name']
        return result
    
    @strawberry.field(description="""user id""")
    async def id(self, info: strawberry.types.Info) -> str:
        result = self['id']
        return result

    @strawberry.field(description="""user id""")
    async def surname(self, info: strawberry.types.Info) -> str:
        result = self['surname']
        return result

    @strawberry.field(description="""group""")
    async def group(self, info: strawberry.types.Info) -> 'GroupGQLModel':
        result = await GroupGQLModel.resolve_reference(info, '123')
        return result   


@strawberry.federation.type(keys=["id"], description="""Entity representing an user""")
class GroupGQLModel:
    @classmethod
    async def resolve_reference(cls, info: strawberry.types.Info, id: strawberry.ID):
        result = {'name': f'23-5-KB({id})', 'id': id}
        return result
    
    @strawberry.field(description="""user name""")
    async def name(self, info: strawberry.types.Info) -> str:
        result = self['name']
        return result
    
    @strawberry.field(description="""user id""")
    async def id(self, info: strawberry.types.Info) -> str:
        result = self['id']
        return result

    @strawberry.field(description="""members""")
    async def users(self, info: strawberry.types.Info) -> List[UserGQLModel]:
        result = []
        return result

@strawberry.type(description="""Type for query root""")
class Query:

    @strawberry.field(description="""Returns the user""")
    async def user_by_id(self, info: strawberry.types.Info, id: strawberry.ID) -> UserGQLModel:
        result = await UserGQLModel.resolve_reference(info, id)
        return result
    
    @strawberry.field(description="""Returns the group""")
    async def group_by_id(self, info: strawberry.types.Info, id: strawberry.ID) -> 'GroupGQLModel':
        result = await GroupGQLModel.resolve_reference(info, id)
        return result
    
from strawberry.asgi import GraphQL

graphql_app = GraphQL(
    strawberry.federation.Schema(Query), 
    graphiql = True,
    allow_queries_via_get = True
)

from fastapi import FastAPI
app = FastAPI()
app.mount("/gql", graphql_app)

start_api(app, port=9992, runNew=True)

INFO:     Started server process [12416]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9992 (Press CTRL+C to quit)


INFO:     172.18.0.1:55260 - "POST / HTTP/1.1" 200 OK
